In [1]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
import glob
import pandas as pd
import numpy as np
import re
import plotly_express as px
import datetime
import warnings
import os
import json
import traceback
import streamlit as st
st.set_page_config(layout="wide")
pd.options.display.float_format = "{:,.1f}".format
warnings.filterwarnings('ignore')


# TEAMNAMES value in teamnamedict must match the htdatan teamname!

global teamnamedict
# C:\Users\mg\JupyterLabDir\Rest\Pr Winning\teamnamedict_streamlit.json
with open('teamnamedict_streamlit.json') as f:
    teamnamedict = json.load(f)

# def _max_width_():
#     max_width_str = f"max-width: 1300px;"
#     st.markdown(
#         f"""
#     <style>
#     .reportview-container .main .block-container{{
#         {max_width_str}
#     }}
#     </style>
#     """,
#         unsafe_allow_html=True,
#     )

# _max_width_()
global widthfig
widthfig = 700
heightfig = 600

# get all the gamestatistics from in dropdown specified league and season
# setup the database connection.  There's no need to setup cursors with pandas psql.
tables = list(glob.glob("htdatan/*"))

# take only the 0 part of the every list entry
global saissons
saissons = []

# ENV is BTCPRED
for x in range(0, len(tables)):    # CHANGE THIS - \\ - to - / - FOR DEPLOYMENT!
    saissons.append(tables[x].split("\\")[1].split("_24102021.csv")[0])


cleaned_names_saissons = []
for saisson in saissons:
    saisson = saisson.replace("_", " ")
    saisson = saisson.replace(".csv", "")
    # saison = saison.strip()
    cleaned_names_saissons.append(saisson)

# map league shortcuts to real names:
shortcut_league_dict = {
    "ll": "La-Liga",
    "pl": "Premier-League",
    "sa": "Serie-A",
    "pl": "Premier-League",
    "l1": "League-1",
    "b": "Bundesliga",
}
# list(map(cleaned_names_saissons, shortcut_league_dict) )
cleaned_names_saissons = [e.replace(
    key, val) for e in cleaned_names_saissons for key, val in shortcut_league_dict.items() if key in e]

# sort list by int substring
cleaned_names_saissons = sorted(cleaned_names_saissons, key=lambda x: int(
    x.split(" ")[-1]), reverse=True)

global saison
saison = "League-1 2122"
print(saison)
# map names back for reading the correct csv name


def find_key(input_dict, value):
    for key, val in input_dict.items():
        if val == value:
            return key
    return "None"


saison = "{}_{}".format(find_key(shortcut_league_dict, saison.split(" ")[0]),
                        saison.split(" ")[1]
                        )

try:
    df_complete_saison = pd.read_csv(
        "htdatan/"+saison+".csv", index_col=0, encoding='utf-8')
except:
    df_complete_saison = pd.read_csv(
        "htdatan/"+saison+"_24102021.csv", index_col=0, encoding='utf-8')

df_complete_saison = df_complete_saison.replace(teamnamedict)
dfallteamnamesl = df_complete_saison.H_Teamnames.unique()

# take only the 0 part of the every list entry
teamsoptions = []
for x in range(0, len(dfallteamnamesl)):
    teamsoptions.append(dfallteamnamesl[x])

global xg_team
xg_team = "Paris SG"

# convert string to df to use process_team_names_of_df function
df_teamname = pd.DataFrame([xg_team])

# convert xg teamnames to correct ones that are used in htdatan
team_df = df_teamname.replace(teamnamedict)
# teamname corrected that it fits to htdatan teamnames
team = team_df.iloc[0][0]

df = df_complete_saison[(df_complete_saison.H_Teamnames == team) | (
    df_complete_saison.A_Teamnames == team)]

# return df, team, saison


def process_team_names_of_df(x_df):
    x_df = x_df.replace(teamnamedict)
    return x_df


League-1 2122


In [4]:

#######################################################
###  calculate table with two halftimes to one game ###
#######################################################
def df_cleaning_converting(df):
    df = df[['H_Teamnames', 'A_Teamnames', 'H_Goals', 'A_Goals', 'H_Ball Possession', 'A_Ball Possession', 'A_Goal Attempts', 'H_Goal Attempts',
             'H_Shots on Goal', 'A_Shots on Goal', 'H_Shots off Goal', 'A_Shots off Goal', 'H_Free Kicks',
             'A_Free Kicks', 'H_Corner Kicks', 'A_Corner Kicks', 'H_Offsides', 'A_Offsides', 'H_Goalkeeper Saves', 'A_Goalkeeper Saves',
             'H_Fouls', 'A_Fouls', 'A_gameinfo', 'A_datetime', 'xG', 'GOALS', 'xPTS', 'A_xG', 'A_GOALS', 'A_xPTS', 'timing_chart_xg']]
    df = df.drop_duplicates(subset=['H_Teamnames', 'A_Teamnames', 'H_Goals', 'A_Goals', 'H_Ball Possession', 'A_Ball Possession', 'A_Goal Attempts', 'H_Goal Attempts',
             'H_Shots on Goal', 'A_Shots on Goal', 'H_Shots off Goal', 'A_Shots off Goal', 'H_Free Kicks',
             'A_Free Kicks', 'H_Corner Kicks', 'A_Corner Kicks', 'H_Offsides', 'A_Offsides', 'H_Goalkeeper Saves', 'A_Goalkeeper Saves',
             'H_Fouls', 'A_Fouls', 'A_gameinfo', 'A_datetime', 'xG', 'GOALS', 'xPTS', 'A_xG', 'A_GOALS', 'A_xPTS'], keep='first')
    df = df.reset_index(drop=True)
    df["R"] = 'X'

    print(df)
    df['timing_chart_xg'] = df['timing_chart_xg'].astype('str') 

    # calculate halftime xG for both teams!
    df['xg_halftime'] = -1
    df['Axg_halftime'] = -1
    for index, row in df.iterrows():
        print(index)
        try:    
            # away team xg at halfime!
            df['Axg_halftime'].loc[index] = df["timing_chart_xg"].loc[index].split("45' ")[0].split("Total xG: ")[-1].split("\n")[0].replace(";", "")  
            # home team xg at halfime!
            df['xg_halftime'].loc[index] = df["timing_chart_xg"].loc[index].split("45' ")[1].split("Total xG: ")[1].split("\n")[0].replace(";", "") 
        except Exception:
            print("WRONG! index: ", index)
            # print(row)
            # print(traceback.format_exc())

    df.xg_halftime = df.xg_halftime.astype(float).fillna(0.0)
    df.Axg_halftime = df.Axg_halftime.astype(float).fillna(0.0) 




    for i in range(0, len(df)):
        try:

            if df["H_Goals"][i] > df["A_Goals"][i]:
                df["R"][i] = 'H'
            if df["H_Goals"][i] < df["A_Goals"][i]:
                df["R"][i] = 'A'
            else:
                df["R"][i] = 'D'
        except:
            print("error?")
    print("!!!", df.columns)
    df.columns = ['Home', 'Opponent', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A',
                'SoG-H', 'SoG-A', 'SoffG-H', 'SoffG-A', 'FK-H',
                'FK-A', 'C-H', 'C-A', 'Off-H', 'Off-A', 'GoKeSa-H', 'GoKeSa-A',
                'F-H', 'F-A', 'Round', 'Date', 'xG', 'GOALS', 'xPTS', 'A_xG', 'A_GOALS', 'A_xPTS', "timing_chart_xg", 'R',  'xg_halftime', 'Axg_halftime',]

    df = df[['Home', 'Opponent', 'R', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A',
             'SoG-H', 'SoG-A', 'SoffG-H', 'SoffG-A', 'FK-H',
             'FK-A', 'C-H', 'C-A', 'Off-H', 'Off-A', 'GoKeSa-H', 'GoKeSa-A', 'F-H',
             'F-A', 'Round', 'Date', 'xG', 'GOALS', 'xPTS', 'A_xG', 'A_GOALS', 'A_xPTS', 'xg_halftime', 'Axg_halftime',]]

    df["IsHome"] = 0

    df = df[['Home', 'Opponent', 'R', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A',
             'SoG-H', 'SoG-A', 'SoffG-H', 'SoffG-A', 'FK-H',
             'FK-A', 'C-H', 'C-A', 'Off-H', 'Off-A', 'GoKeSa-H', 'GoKeSa-A', 'F-H',
             'F-A', 'Round', 'Date', 'IsHome', 'xG', 'GOALS', 'xPTS', 'A_xG', 'A_GOALS', 'A_xPTS','xg_halftime', 'Axg_halftime',]]
    return df


 
def df_specific_team(df, team):
    df4Home = df.loc[((df['Home'] == team))]

    df4Home["IsHome"] = 1

    # recalcualte the winner because of the columns switching to bring the selected team in the first column
    df4Home["1x2"] = 0

    df4Home["1x2"] = df4Home.apply(
        lambda row: calculate_1x2_home(row), axis=1, result_type='reduce')

    # Berechnung Opponentgames
    df4Opponent = df.loc[((df['Opponent'] == team))]


    

    # recalcualte the winner because of the columns switching to bring the selected team in the first column
    df4Opponent["1x2"] = 0

    df4Opponent["1x2"] = df4Opponent.apply(
        lambda row: calculate_1x2_Opponent(row), axis=1, result_type='reduce')

    # change the halftime-xg with halftime-Axg:
    # switched the two columns
    # Change the columns for the Opponentmatches of the specific team
    OpponentTeamReversedColumns = ['Opponent', 'Home',  '1x2', 'R',  'G-A', 'G-H',
                                   'BP-A', 'BP-H', 'GA-A', 'GA-H',  'SoG-A', 'SoG-H', 'SoffG-A', 'SoffG-H',  'FK-A', 'FK-H',
                                   'C-A', 'C-H',  'Off-A', 'Off-H', 'GoKeSa-A', 'GoKeSa-H', 'F-A', 'F-H',
                                   'Round', 'Date', 'IsHome',
                                   'A_xG', "A_xPTS", "A_GOALS", 'xG', "xPTS", "GOALS", 'Axg_halftime', 'xg_halftime' ]  # , 'IsHome'

    df4OpponentReversed = df4Opponent.reindex(
        columns=OpponentTeamReversedColumns)

    df4OpponentReversed.columns = ['Home', 'Opponent', '1x2', 'R', 'G-H',
                                   'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A',
                                   'SoG-H', 'SoG-A', 'SoffG-H', 'SoffG-A', 'FK-H',
                                   'FK-A', 'C-H', 'C-A', 'Off-H', 'Off-A', 'GoKeSa-H', 'GoKeSa-A', 'F-H',
                                   'F-A', 'Round', 'Date', 'IsHome',
                                   'xG', "xPTS", "GOALS", 'A_xG', "A_xPTS", "A_GOALS", 'xg_halftime', 'Axg_halftime',]
    return df4Home, df4OpponentReversed


 
def create_df4Complete(df4Home, df4OpponentReversed):
    # Alle Spiele werden als Heimspiel angezeigt, sind aber auch Auswärtsspiele dabei!
    df4Complete = pd.concat([df4Home, df4OpponentReversed], sort=False)

    df4Complete["G-H"] = df4Complete["G-H"].astype('float64')
    df4Complete["G-A"] = df4Complete["G-A"].astype('float64')
    df4Complete["BP-H"] = df4Complete["BP-H"].astype('float64')
    df4Complete["BP-A"] = df4Complete["BP-A"].astype('float64')

    # GoalDifference
    df4Complete["GoalDiff"] = df4Complete["G-H"] - df4Complete["G-A"]
    df4Complete = df4Complete.sort_values("Date",  ascending=False)

    # calculate column with 3 Ballposition types
    df4Complete["BPTypes"] = '0'

    df4Complete["BPTypes"] = df4Complete.apply(
        lambda row: calculate_1x2_BPTypes(row), axis=1, result_type='reduce')

    df4Complete['Date'] = pd.to_datetime(
        df4Complete['Date'], format="%d.%m.%Y %H:%M")

    # convert datetime to timestamp for scatter visualization
    df4Complete['timestamp'] = df4Complete.Date.astype('int64')//10**9

    df4Complete = df4Complete.sort_values("Date", ascending=False)

    # Create data for scatter graph
    df4Complete["SoG-H"] = df4Complete["SoG-H"].astype(int)
    df4Complete["SoG-A"] = df4Complete["SoG-A"].astype(int)

    return df4Complete


 
def calculate_1x2_home(row):
    if row['G-H'] > row['G-A']:
        return 'W'
    elif row['G-A'] > row['G-H']:
        return 'L'
    else:
        return 'D'


 
def calculate_1x2_Opponent(row):
    if row['G-A'] > row['G-H']:
        return 'W'
    elif row['G-H'] > row['G-A']:
        return 'L'
    else:
        return 'D'


 
def calculate_1x2_BPTypes(row):
    if row['BP-H'] > 55:
        return '>55'
    elif row['BP-H'] < 45:
        return '<45'
    else:
        return '45-55'


 
def calc_stats(df4Complete):
    BP_WPerc = df4Complete[['BPTypes', '1x2']
                           ].loc[df4Complete['BPTypes'] == '>55']
    BP_WAbs = BP_WPerc['1x2'].loc[df4Complete['1x2'] == 'W']
    BP_NWAbs = BP_WPerc['1x2'].loc[df4Complete['1x2'] != 'W']
    if len(BP_WAbs) + len(BP_NWAbs) > 0:
        BP_WPercText = len(BP_WAbs) / (len(BP_WAbs) + len(BP_NWAbs)) * 100
        BP_WPercText = round(BP_WPercText)
    else:
        BP_WPercText = 0

    # calculate winning % for 0.45 - 0.55
    N_WPerc = df4Complete[['BPTypes', '1x2']
                          ].loc[df4Complete['BPTypes'] == '45-55']
    N_WAbs = N_WPerc['1x2'].loc[df4Complete['1x2'] == 'W']
    N_NWAbs = N_WPerc['1x2'].loc[df4Complete['1x2'] != 'W']
    if len(N_WAbs) + len(N_NWAbs) > 0:
        N_WPercText = len(N_WAbs) / (len(N_WAbs) + len(N_NWAbs)) * 100
        N_WPercText = round(N_WPercText)
    else:
        N_WPercText = 5

    # calculate winning % for < 0.45
    C_WPerc = df4Complete[['BPTypes', '1x2']
                          ].loc[df4Complete['BPTypes'] == '<45']
    C_WAbs = C_WPerc['1x2'].loc[df4Complete['1x2'] == 'W']
    C_NWAbs = C_WPerc['1x2'].loc[df4Complete['1x2'] != 'W']
    if len(C_WAbs) + len(C_NWAbs) > 0:
        C_WPercText = len(C_WAbs) / (len(C_WAbs) + len(C_NWAbs)) * 100
        C_WPercText = round(C_WPercText)
    else:
        C_WPercText = 0

    return C_WPercText, N_WPercText, BP_WPercText


# get name of the selected team in dropdown
def load_xg_gamestats_sql(saison, team):

    if saison.split("_")[0] == 'b':
        xgprefix = 'bundesliga'
    elif saison.split("_")[0] == 'l1':
        xgprefix = 'ligue_1'
    elif saison.split("_")[0] == 'll':
        xgprefix = 'la_liga'
    elif saison.split("_")[0] == 'pl':
        xgprefix = 'epl'
    elif saison.split("_")[0] == 'sa':
        xgprefix = 'serie_a'

    xgtablename = "{}20{}".format(xgprefix, saison.split("_")[1][:2])

    df_complete_saison = pd.read_csv(
        "xg/"+xgtablename+".csv", index_col=0, encoding='utf-8')

    df_complete_saison = process_team_names_of_df(df_complete_saison)

    # execute the query and assign it to a pandas dataframe
    dfxg = df_complete_saison[(df_complete_saison.TEAMS == teamx) | (
        df_complete_saison.A_TEAMS == teamx)]

    

    return dfxg


df = process_team_names_of_df(df)
df

,H_datetime,H_gameinfo,H_Teamnames,H_Goals,H_Ball Possession,H_Goal Attempts,H_Shots on Goal,H_Shots off Goal,H_Blocked Shots,H_Free Kicks,...,A_Fouls,A_Yellow Cards,A_Red Cards,A_Total Passes,A_Completed Passes,A_Tackles,A_Attacks,A_Dangerous Attacks,A_Throw-in,H_cat
4,07.08.2021 12:00,FootballFootballFRANCE: Ligue 1 - Round 1,Troyes,0,44,10,3,3,4,13,...,9,2.0,NaN,313,296.0,5,66,42,6.0,cat4
5,07.08.2021 12:00,FootballFootballFRANCE: Ligue 1 - Round 1,Troyes,1,42,5,4,1,0,7,...,6,2.0,NaN,292,264.0,5,74,32,6.0,cat5
24,14.08.2021 12:00,FootballFootballFRANCE: Ligue 1 - Round 2,Paris SG,1,62,5,3,2,0,14,...,9,4.0,1.0,190,163.0,7,30,18,8.0,cat24
25,14.08.2021 12:00,FootballFootballFRANCE: Ligue 1 - Round 2,Paris SG,3,68,6,3,1,2,14,...,9,NaN,NaN,174,146.0,9,34,10,4.0,cat25
40,20.08.2021 12:00,FootballFootballFRANCE: Ligue 1 - Round 3,Brest,1,29,8,2,4,2,7,...,4,2.0,NaN,379,347.0,7,68,24,12.0,cat40
41,20.08.2021 12:00,FootballFootballFRANCE: Ligue 1 - Round 3,Brest,1,23,3,1,1,1,3,...,3,NaN,NaN,440,412.0,9,80,34,8.0,cat41
76,29.08.2021 11:45,FootballFootballFRANCE: Ligue 1 - Round 4,Reims,0,32,9,4,0,5,7,...,7,2.0,NaN,351,311.0,10,61,12,12.0,cat76
77,29.08.2021 11:45,FootballFootballFRANCE: Ligue 1 - Round 4,Reims,0,34,2,0,2,0,6,...,6,1.0,NaN,375,334.0,8,76,42,9.0,cat77
80,11.09.2021 08:00,FootballFootballFRANCE: Ligue 1 - Round 5,Paris SG,2,45,6,3,2,1,5,...,2,NaN,NaN,319,283.0,10,37,22,10.0,cat80
81,11.09.2021 08:00,FootballFootballFRANCE: Ligue 1 - Round 5,Paris SG,2,59,7,4,1,2,8,...,7,1.0,NaN,210,177.0,10,49,22,8.0,cat81


In [5]:
dfxg = load_xg_gamestats_sql(saison, team)
dfxg

,TEAMS,GOALS,xG,SHOTS,SHOTS ON TARGET,DEEP,PPDA,xPTS,A_TEAMS,A_GOALS,A_xG,A_SHOTS,A_SHOTS ON TARGET,A_DEEP,A_PPDA,A_xPTS,timing_chart_xg,id
12,Paris SG,2,1.9,9,2,4,16.5,2.3,Monaco,0,0.8,13,3,6,7.6,0.5,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,13
38,Paris SG,3,1.3,14,7,9,12.7,1.6,Nantes,1,1.0,12,6,3,22.0,1.1,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,40
56,Paris SG,2,2.2,14,5,9,8.2,2.5,Lyon,1,0.8,12,5,2,12.7,0.4,0' Total xG: 0.00;0' Total xG: 0.00;1' Total x...,60
63,Lyon,1,1.0,11,4,4,20.2,1.3,Paris SG,1,1.1,15,3,6,6.7,1.4,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,69
75,Paris SG,2,2.7,20,9,16,7.6,2.6,Brest,0,0.7,6,3,0,40.5,0.2,0' Total xG: 0.00;0' Total xG: 0.00;1' Total x...,83
85,Brest,2,1.2,11,3,5,34.6,1.1,Paris SG,4,1.5,16,8,14,8.1,1.6,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,93
93,Troyes,1,0.6,15,7,6,66.8,0.6,Paris SG,2,1.5,12,3,12,10.8,2.1,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,106
106,Paris SG,2,2.6,16,5,10,6.6,2.8,Montpellier,0,0.3,10,4,1,25.6,0.1,0' Total xG: 0.09;0' Total xG: 0.00;1' Total x...,121
107,Paris SG,2,2.3,17,6,13,12.9,1.7,Lille,1,1.8,11,3,12,14.9,1.1,0' Total xG: 0.00;0' Total xG: 0.27;1' Total x...,122
108,Paris SG,4,3.0,22,8,12,9.5,2.9,Reims,0,0.3,7,3,3,18.5,0.1,0' Total xG: 0.00;0' Total xG: 0.00;1' Total x...,123


In [6]:



# rename columns for
dfxg_rename = dfxg.rename(
    columns={'TEAMS': 'H_Teamnames', 'A_TEAMS': 'A_Teamnames'})
# del dfxg

dfxg_df_merged = pd.merge(
    df, dfxg_rename, on=["H_Teamnames", "A_Teamnames"])
dfxg_df_merged = dfxg_df_merged.drop_duplicates()

# %%
df = dfxg_df_merged

In [ ]:
asdasd

In [ ]:


dfxg_df_merged_cleaned = df_cleaning_converting(df)

df4Home, df4OpponentReversed = df_specific_team(
    dfxg_df_merged_cleaned, team)

df4Complete = create_df4Complete(df4Home, df4OpponentReversed)

slidertext = 'Show last x halftimes'
nrGames = st.sidebar.slider(slidertext, max_value=len(
    df4Complete), value=len(df4Complete), step=2)

# change rows of df depending on userinput
df4Complete = df4Complete[:nrGames]
df4Complete = df4Complete.sort_values("Date", ascending=False)
df4Complete = df4Complete.round(1)

df4Complete[['xG', 'A_xG', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A', 'SoG-H', 'SoG-A',  'xPTS', 'A_xPTS']] = df4Complete[['xG',
                                                                                                                              'A_xG', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A', 'SoG-H', 'SoG-A',  'xPTS', 'A_xPTS']].apply(pd.to_numeric, errors='coerce', axis=1)

df4Complete_show = df4Complete[['Home', 'Opponent', 'IsHome', 'R', 'xG', 'A_xG', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A',
                                'SoG-H', 'SoG-A',  'xPTS', 'A_xPTS', 'Date', 'xg_halftime', 'Axg_halftime',]]


# create df for visualizing
df4CompleteGraph = df4Complete.copy()

df4CompleteGraph["SoG-H-SoG-A"] = df4CompleteGraph["SoG-H"] - \
    df4CompleteGraph["SoG-A"]
df4CompleteGraph["SoG-H-SoG-A"] = df4CompleteGraph["SoG-H-SoG-A"].clip(
    lower=0)

df4CompleteGraph["SoG-A-SoG-H"] = df4CompleteGraph["SoG-A"] - \
    df4CompleteGraph["SoG-H"]
df4CompleteGraph["SoG-A-SoG-H"] = df4CompleteGraph["SoG-A-SoG-H"].clip(
    lower=0)

figScatter = px.scatter(
    df4CompleteGraph,  # .query(f'Date.between{end_date}'),
    x='BP-H',
    y='GoalDiff',
    marginal_x="histogram",
    color="timestamp",
    size="SoG-H-SoG-A",
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    # title="SoGH-SoGA - Halftimes",
    # color_continuous_scale= 'Viridis',
    # color_discrete_map={"W": "green", "D": "gray", "L": "red"}

    # facet_row="time", # makes seperate plot for value
    # marginal_x="histogram",
).update_traces(textposition='top center', marker_symbol="cross", selector={'type': 'scatter'}).update_traces(
    marker=dict(color='green'), selector={'type': 'histogram'}
)
figScatter.update_xaxes(range=[5, 95])
figScatter.update_layout(
    title_text='All halftimes: Shots on Goal - Shots on Goal Opponent', title_x=0.5,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))

figScatter1 = px.scatter(
    df4CompleteGraph,  # .query(f'Date.between{end_date}'),
    x='BP-H',
    y='GoalDiff',
    marginal_x="histogram",
    color="timestamp",
    size="SoG-A-SoG-H",
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    title="SoGA-SoGH - Halftimes",
    # color_discrete_map={"W": "green", "D": "gray", "L": "red"}

    # facet_row="time", # makes seperate plot for value
    # marginal_x="histogram",
).update_traces(
    textposition='top center', selector={'type': 'scatter'}).update_traces(
        marker=dict(color='red'), selector={'type': 'histogram'}
)
figScatter1.update_xaxes(range=[5, 95])
figScatter1.update_layout(
    title_text='All halftimes: Shots on Goal Opponent - Shots on Goal', title_x=0.5,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    )
)


# wie oft passierts das team hinten ist und noch gewinnt / nicht verliert:´
# sortier nach sieg x loss
# mach diagramm halftimes für 1te und 2te hz

# delete games where there is no two halftimes!
df4CompleteGraph = df4CompleteGraph[df4CompleteGraph.groupby(
    'Opponent')['Opponent'].transform('size') >= 2]
df4CompleteGraph = df4CompleteGraph.sort_index()
# second half is second entry always!
df4CompleteGraph["halftime"] = "0"
df4CompleteGraph.iloc[::2]["halftime"] = "2"
df4CompleteGraph.iloc[1::2]["halftime"] = "1"

figScatter5 = px.scatter(
    # .query(f'Date.between{end_date}'),
    df4CompleteGraph[df4CompleteGraph["halftime"] == "1"],
    x='BP-H',
    y='GoalDiff',
    size="SoG-H-SoG-A",
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    # color_continuous_scale= 'Viridis',
    # facet_row="time", # makes seperate plot for value
    marginal_x="histogram",
).update_traces(textposition='top center', marker_symbol="cross", marker=dict(
    color='green'), selector={'type': 'scatter'}
).update_traces(marker=dict(
    color='green'), selector={'type': 'histogram'}
)

figScatter5.update_xaxes(range=[5, 95])
figScatter5.update_layout(
    title_text='Halftime 1: Shots on Goal - Shots on Goal Opponent', title_x=0.5,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))


figScatter6 = px.scatter(
    # .query(f'Date.between{end_date}'),
    df4CompleteGraph[df4CompleteGraph["halftime"] == "1"],
    x='BP-H',
    y='GoalDiff',
    size="SoG-A-SoG-H",
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    # color_continuous_scale= 'Viridis',
    # facet_row="time", # makes seperate plot for value
    marginal_x="histogram",
).update_traces(textposition='top center', marker=dict(
    color='red'), selector={'type': 'scatter'}
).update_traces(marker=dict(
    color='red'), selector={'type': 'histogram'}
)

figScatter6.update_xaxes(range=[5, 95])
figScatter6.update_layout(
    title_text='Halftime 1: Shots on Goal Opponent - Shots on Goal', title_x=0.5,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))


figScatter7 = px.scatter(
    # .query(f'Date.between{end_date}'),
    df4CompleteGraph[df4CompleteGraph["halftime"] == "2"],
    x='BP-H',
    y='GoalDiff',
    marginal_x="histogram",
    size="SoG-H-SoG-A",
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    # color_continuous_scale= 'Viridis',
    # facet_row="time", # makes seperate plot for value

).update_traces(textposition='top center', marker_symbol="cross", marker=dict(
    color='green'), selector={'type': 'scatter'}
).update_traces(marker=dict(
    color='green'), selector={'type': 'histogram'}
)

figScatter7.update_xaxes(range=[5, 95])
figScatter7.update_layout(
    title_text='Halftime 2: Shots on Goal - Shots on Goal Opponent', title_x=0.5,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    )
)


figScatter8 = px.scatter(
    # .query(f'Date.between{end_date}'),
    df4CompleteGraph[df4CompleteGraph["halftime"] == "2"],
    x='BP-H',
    y='GoalDiff',
    marginal_x="histogram",
    size="SoG-A-SoG-H",
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    # color_continuous_scale= 'Viridis',
    # facet_row="time", # makes seperate plot for value
    # marginal_x="histogram",
).update_traces(textposition='top center', marker=dict(
    color='red'), selector={'type': 'scatter'}
).update_traces(marker=dict(
    color='red'), selector={'type': 'histogram'}
)


figScatter8.update_xaxes(range=[5, 95])
figScatter8.update_layout(
    title_text='Halftime 2: Shots on Goal Opponent - Shots on Goal', title_x=0.5,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))


naming1x2 = {"W": "Win", "D": "Draw", "L": "Loss"}
df4CompleteGraph['Halftime result'] = df4CompleteGraph['1x2'].replace(
    naming1x2)

try:
    highest_count_yaxis = df4CompleteGraph.groupby(["BPTypes", "halftime"]).agg(
        'count').sort_values("Opponent", ascending=False).iloc[0].Home
except:
    highest_count_yaxis = 0


# Create data for histogram 2
BarBallpossesionstylesResultsHalftime1 = px.bar(
    df4CompleteGraph[df4CompleteGraph["halftime"] == "1"],
    x='BPTypes',
    # text=df4CompleteGraph.index,
    # title="BP-Styles - Halftimes",
    color='Halftime result',
    color_discrete_map={"Win": "green", "Draw": "gray", "Loss": "red"},
    width=widthfig,
    # height=heightfig,
    # opacity=0.5,
    text="Opponent",
).update_xaxes(categoryorder="array", categoryarray=['<45', '45-55', '>55'],).update_yaxes(
    range=[0, highest_count_yaxis])

BarBallpossesionstylesResultsHalftime1.update_layout(
    title_text='Ballpossesionstyles - results halftime 1', title_x=0.5, xaxis=dict(
        tickmode='array', showticklabels=True,
    )
)

# Create data for histogram 2
BarBallpossesionstylesResultsHalftime2 = px.bar(
    df4CompleteGraph[df4CompleteGraph["halftime"] == "2"],
    x='BPTypes',
    # text=df4CompleteGraph.index,
    # title="BP-Styles - Halftimes",
    color='Halftime result',
    color_discrete_map={"Win": "green", "Draw": "gray", "Loss": "red"},
    width=widthfig,
    # height=heightfig,
    # opacity=0.5,
    text="Opponent",
).update_xaxes(categoryorder="array", categoryarray=['<45', '45-55', '>55']).update_yaxes(
    range=[0, highest_count_yaxis])
BarBallpossesionstylesResultsHalftime2.update_layout(
    title_text='Ballpossesionstyles - results halftime 2', title_x=0.5, xaxis=dict(
        tickmode='array', showticklabels=True,
    )
)


# # create scatterplot with XG - bubble size
# df4CompleteGraph = df4Complete.copy()

# df4CompleteGraph = convert_hts_to_complete_games(df4CompleteGraph)

# # Calculate again the stuff like for the single halftimes before!
# # GoalDifference
# df4CompleteGraph["GoalDiff"] = df4CompleteGraph["G-H"] - \
#     df4CompleteGraph["G-A"]
# df4CompleteGraph = df4CompleteGraph.sort_values("Date",  ascending=False)
# # calculate column with 3 Ballposition types
# df4CompleteGraph["BPTypes"] = '0'
# df4CompleteGraph["BPTypes"] = df4CompleteGraph.apply(
#     lambda row: calculate_1x2_BPTypes(row), axis=1, result_type='reduce')
# df4CompleteGraph['Date'] = pd.to_datetime(
#     df4CompleteGraph['Date'], format="%d.%m.%Y %H:%M")
# # convert datetime to timestamp for scatter visualization
# df4CompleteGraph['timestamp'] = df4CompleteGraph.Date.astype('int64')//10**9
# df4CompleteGraph = df4CompleteGraph.sort_values("Date", ascending=False)
# Create data for scatter graph
print(df4CompleteGraph.columns)
print(df4CompleteGraph[["xG","A_xG", "xg_halftime", "Axg_halftime","halftime","Opponent",'Halftime result',"timestamp"]])

df4CompleteGraph.xg_halftime = df4CompleteGraph.xg_halftime.astype(float).fillna(0.0)
df4CompleteGraph.Axg_halftime = df4CompleteGraph.Axg_halftime.astype(float).fillna(0.0) 

# all xg values for both halftimes!
df4CompleteGraph["xG-A_xG"] = df4CompleteGraph["xG"] - df4CompleteGraph["A_xG"]
df4CompleteGraph["A_xG-xG"] = df4CompleteGraph["A_xG"] - df4CompleteGraph["xG"]
# all values for first half
df4CompleteGraph["xg_halftime-Axg_halftime"] = df4CompleteGraph["xg_halftime"] - df4CompleteGraph["Axg_halftime"]
df4CompleteGraph["Axg_halftime-xg_halftime"] = df4CompleteGraph["Axg_halftime"] - df4CompleteGraph["xg_halftime"]
# all values for second half
df4CompleteGraph["xg_halftime2-Axg_halftime2"] = df4CompleteGraph["xG-A_xG"] - df4CompleteGraph["xg_halftime-Axg_halftime"]
df4CompleteGraph["xg_halftime2-Axg_halftime2"] = df4CompleteGraph["xg_halftime2-Axg_halftime2"].clip(lower=0)
df4CompleteGraph["xg_halftime2-Axg_halftime2"] = df4CompleteGraph["xg_halftime2-Axg_halftime2"].round(2)
df4CompleteGraph["Axg_halftime2-xg_halftime2"] = df4CompleteGraph["xg_halftime-Axg_halftime"] - df4CompleteGraph["xG-A_xG"]
df4CompleteGraph["Axg_halftime2-xg_halftime2"] = df4CompleteGraph["Axg_halftime2-xg_halftime2"].clip(lower=0)
df4CompleteGraph["Axg_halftime2-xg_halftime2"] = df4CompleteGraph["Axg_halftime2-xg_halftime2"].round(2)
df4CompleteGraph["xG-A_xG"] = df4CompleteGraph["xG-A_xG"].clip(lower=0)
df4CompleteGraph["xG-A_xG"] = df4CompleteGraph["xG-A_xG"].round(2)
df4CompleteGraph["A_xG-xG"] = df4CompleteGraph["A_xG-xG"].clip(lower=0)
df4CompleteGraph["A_xG-xG"] = df4CompleteGraph["A_xG-xG"].round(2)
df4CompleteGraph["xg_halftime-Axg_halftime"] = df4CompleteGraph["xg_halftime-Axg_halftime"].clip(lower=0)
df4CompleteGraph["xg_halftime-Axg_halftime"] = df4CompleteGraph["xg_halftime-Axg_halftime"].round(2)
df4CompleteGraph["Axg_halftime-xg_halftime"] = df4CompleteGraph["Axg_halftime-xg_halftime"].clip(lower=0)
df4CompleteGraph["Axg_halftime-xg_halftime"] = df4CompleteGraph["Axg_halftime-xg_halftime"].round(2)


print(df4CompleteGraph[["xG","A_xG", "xg_halftime", "Axg_halftime","halftime","Opponent",'Halftime result',"timestamp"]])
ht1 = df4CompleteGraph[df4CompleteGraph["halftime"] == "1"]
print(ht1[["IsHome","xG","A_xG", "xg_halftime", "Axg_halftime","halftime","Opponent",'Halftime result',"timestamp"]])
ht2 = df4CompleteGraph[df4CompleteGraph["halftime"] == "2"]
print(ht2[["IsHome","xG","A_xG", "xg_halftime", "Axg_halftime","halftime","Opponent",'Halftime result',"timestamp"]])

figHistogramxG_A_xG_1Ht = px.scatter(
    df4CompleteGraph[df4CompleteGraph["halftime"] == "1"],  # .query(f'Date.between{end_date}'),
    x='BP-H',
    y='GoalDiff',
    marginal_x="histogram",
    color="timestamp",
    size="xg_halftime-Axg_halftime",
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    # color_continuous_scale= 'Viridis',
    # facet_row="time", # makes seperate plot for value
    # marginal_x="histogram",
).update_traces(textposition='top center', marker_symbol="cross", selector={'type': 'scatter'} ).update_traces(
    marker=dict(color='green'), selector={'type': 'histogram'}
)
figHistogramxG_A_xG_1Ht.update_xaxes(range=[5, 95])
figHistogramxG_A_xG_1Ht.update_layout(
    title_text='Ht1: Expectedgoals - Expectedgoals Opponent', title_x=0.5,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))

figHistogramA_xG_xG_1Ht = px.scatter(
    df4CompleteGraph[df4CompleteGraph["halftime"] == "1"],  # .query(f'Date.between{end_date}'),
    x='BP-H',
    y='GoalDiff',
    marginal_x="histogram",
    color="timestamp",
    size="Axg_halftime-xg_halftime",
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    # facet_row="time", # makes seperate plot for value
    # marginal_x="histogram",
).update_traces(textposition='top center', selector={'type': 'scatter'} ).update_traces(
    marker=dict(color='red'), selector={'type': 'histogram'}
)
figHistogramA_xG_xG_1Ht.update_xaxes(range=[5, 95])
figHistogramA_xG_xG_1Ht.update_layout(
    title_text='Ht1: Expectedgoals Opponent - Expectedgoals', title_x=0.5,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))


figHistogramxG_A_xG_2Ht = px.scatter(
    df4CompleteGraph[df4CompleteGraph["halftime"] == "2"],  # .query(f'Date.between{end_date}'),
    x='BP-H',
    y='GoalDiff',
    marginal_x="histogram",
    color="timestamp",
    size="xg_halftime2-Axg_halftime2",
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    # color_continuous_scale= 'Viridis',
    # facet_row="time", # makes seperate plot for value
    # marginal_x="histogram",
).update_traces(textposition='top center', marker_symbol="cross", selector={'type': 'scatter'} ).update_traces(
    marker=dict(color='green'), selector={'type': 'histogram'}
)

figHistogramxG_A_xG_2Ht.update_xaxes(range=[5, 95])
figHistogramxG_A_xG_2Ht.update_layout(
    title_text='Ht2: Expectedgoals - Expectedgoals Opponent', title_x=0.5,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))

figHistogramA_xG_xG_2Ht = px.scatter(
    df4CompleteGraph[df4CompleteGraph["halftime"] == "2"],  # .query(f'Date.between{end_date}'),
    x='BP-H',
    y='GoalDiff',
    marginal_x="histogram",
    color="timestamp",
    size="Axg_halftime2-xg_halftime2",
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    # facet_row="time", # makes seperate plot for value
    # marginal_x="histogram",
).update_traces(textposition='top center', selector={'type': 'scatter'} ).update_traces(
    marker=dict(color='red'), selector={'type': 'histogram'}
)

figHistogramA_xG_xG_2Ht.update_xaxes(range=[5, 95])
figHistogramA_xG_xG_2Ht.update_layout(
    title_text='Ht2: Expectedgoals Opponent - Expectedgoals', title_x=0.5,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))




# Streamlit encourages well-structured code, like starting execution in a main() function.
st.title("Football statistics - {}".format(team))
st.markdown('The following two diagrams display the new metric Expected Goals (**xGoals**), which is a qualitative measurement on base of the shots on goal.  \nThe expected goal model shows how high the chance of the goal really was and calculates a value for each completion based on several factors.   \nF.I. a penalty has generally a probably of 75 % to result in a goal, which would increase the xGoal value for 0.75 regardless of the penalty-outcame in this case.', unsafe_allow_html=False)

col1, col2 = st.columns(2)

col1.plotly_chart(figHistogramxG_A_xG_1Ht)

col2.plotly_chart(figHistogramA_xG_xG_1Ht)

col1.plotly_chart(figHistogramxG_A_xG_2Ht)

col2.plotly_chart(figHistogramA_xG_xG_2Ht)

col1.plotly_chart(figScatter)

col2.plotly_chart(figScatter1)

col1.plotly_chart(figScatter5)

col2.plotly_chart(figScatter6)

col1.plotly_chart(figScatter7)

col2.plotly_chart(figScatter8)

col1.plotly_chart(BarBallpossesionstylesResultsHalftime1)

col2.plotly_chart(BarBallpossesionstylesResultsHalftime2)

# C_WPercText, N_WPercText, BP_WPercText = calc_stats(df4Complete)
# col2.write("% W < 0.45:   {}   \n % W 0.45 - 0.55:  {}   \n % W > 0.55:  {}".format(
#     C_WPercText, N_WPercText, BP_WPercText))

# show df
st.dataframe(df4Complete_show.style.format({'xG': '{:.1f}', 'A_xG': '{:.1f}', 'SoG-H': '{:.0f}',
                                            'G-H': '{:.0f}', 'G-A': '{:.0f}', 'BP-H': '{:.0f}',
                                            'BP-A': '{:.0f}', 'GA-H': '{:.0f}', 'GA-A': '{:.0f}',
                                            'xPTS': '{:.1f}', 'A_xPTS': '{:.1f}', 'SoG-A': '{:.0f}',
                                            }))
